<a href="https://colab.research.google.com/github/remotesensinginfo/pb_gee_tools/blob/main/examples/extract_data/Extract_GRIDMET_Drought.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# The Google Earth Engine module
import ee

# The datetime module is used to specify the dates
# to search for imagery
import datetime

# Geopandas allows us to read the shapefile used to
# define the region of interest (ROI)
import geopandas

# Pandas allows us to create a spreadsheet output
# datasets
import pandas

# The colab module to access data from your google drive
from google.colab import drive

In [ ]:
try:
  import pb_gee_tools.convert_types
except:
  !git clone https://github.com/remotesensinginfo/pb_gee_tools.git
  !pip install ./pb_gee_tools/.
  import pb_gee_tools.convert_types

In [ ]:
ee_prj_name = "ee-pb-dev"  # <==== Replace this with your own EE project string
ee.Authenticate()
ee.Initialize(project=ee_prj_name)

In [ ]:
drive.mount("/content/drive")

In [ ]:
# The region of interest
vec_roi_file = "/content/drive/MyDrive/drought_data_florida_mangroves/south_florida_mangroves.geojson"


# Dates before the fire
start_date = datetime.datetime(year=2020, month=1, day=1)
end_date = datetime.datetime(year=2024, month=5, day=28)

# No Data Value
out_no_data_val = 0.0

In [ ]:
# Read the vector layer and make sure it is project using WGS84 (EPSG:4326)
vec_gdf = geopandas.read_file(vec_roi_file).to_crs(4326)

# Get layer bbox: minx, miny, maxx, maxy
gp_bbox = vec_gdf.total_bounds

# Create the GEE geometry from the bbox.
roi_west = gp_bbox[0]
roi_east = gp_bbox[2]
roi_north = gp_bbox[3]
roi_south = gp_bbox[1]
gee_bbox_aoi = ee.Geometry.BBox(roi_west, roi_south, roi_east, roi_north)

In [ ]:
roi_ee_poly = pb_gee_tools.convert_types.convert_vector_to_gee_polygon(vec_file = vec_roi_file)

In [ ]:
###########################################################
# *** Note this dataset is only available for the USA. ***
###########################################################
drought_img_col = ee.ImageCollection('GRIDMET/DROUGHT').filterBounds(gee_bbox_aoi).filter(ee.Filter.date(start_date, end_date))

In [ ]:
def create_time_series(image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(), geometry=roi_ee_poly)
    return ee.Feature(None, {"date": image.date().format(),
                             "spi14d": stats.get("spi14d"),
                             "spi30d": stats.get("spi30d"),
                             "spi90d": stats.get("spi90d"),
                             "spi180d": stats.get("spi180d"),
                             "spi270d": stats.get("spi270d"),
                             "spi1y": stats.get("spi1y"),
                             "spi2y": stats.get("spi2y"),
                             "spi5y": stats.get("spi5y"),
                             "eddi14d": stats.get("eddi14d"),
                             "eddi30d": stats.get("eddi30d"),
                             "eddi90d": stats.get("eddi90d"),
                             "eddi180d": stats.get("eddi180d"),
                             "eddi270d": stats.get("eddi270d"),
                             "eddi1y": stats.get("eddi1y"),
                             "eddi2y": stats.get("eddi2y"),
                             "eddi5y": stats.get("eddi5y"),
                             "spei14d": stats.get("spei14d"),
                             "spei30d": stats.get("spei30d"),
                             "spei90d": stats.get("spei90d"),
                             "spei180d": stats.get("spei180d"),
                             "spei270d": stats.get("spei270d"),
                             "spei1y": stats.get("spei1y"),
                             "spei2y": stats.get("spei2y"),
                             "spei5y": stats.get("spei5y"),
                             "pdsi": stats.get("pdsi"),
                             "z": stats.get("z"),})

In [ ]:
drought_time_series = drought_img_col.map(create_time_series).getInfo()

In [ ]:
# Convert time series to a pandas DataFrame
drought_feats = drought_time_series["features"]

# Create a list of the dates and drought data
drought_data = list()
for drought_feat in drought_feats:
    if "date" in drought_feat["properties"]:
        drought_data.append(
            {
                "date": drought_feat["properties"]["date"],
                "spi14d": drought_feat["properties"]["spi14d"],
                "spi30d": drought_feat["properties"]["spi30d"],
                "spi90d": drought_feat["properties"]["spi90d"],
                "spi180d": drought_feat["properties"]["spi180d"],
                "spi270d": drought_feat["properties"]["spi270d"],
                "spi1y": drought_feat["properties"]["spi1y"],
                "spi2y": drought_feat["properties"]["spi2y"],
                "spi5y": drought_feat["properties"]["spi5y"],
                "eddi14d": drought_feat["properties"]["eddi14d"],
                "eddi30d": drought_feat["properties"]["eddi30d"],
                "eddi90d": drought_feat["properties"]["eddi90d"],
                "eddi180d": drought_feat["properties"]["eddi180d"],
                "eddi270d": drought_feat["properties"]["eddi270d"],
                "eddi1y": drought_feat["properties"]["eddi1y"],
                "eddi2y": drought_feat["properties"]["eddi2y"],
                "eddi5y": drought_feat["properties"]["eddi5y"],
                "spei14d": drought_feat["properties"]["spei14d"],
                "spei30d": drought_feat["properties"]["spei30d"],
                "spei90d": drought_feat["properties"]["spei90d"],
                "spei180d": drought_feat["properties"]["spei180d"],
                "spei270d": drought_feat["properties"]["spei270d"],
                "spei1y": drought_feat["properties"]["spei1y"],
                "spei2y": drought_feat["properties"]["spei2y"],
                "spei5y": drought_feat["properties"]["spei5y"],
                "pdsi": drought_feat["properties"]["pdsi"],
                "z": drought_feat["properties"]["z"],
            }
        )

# Convert the list of a Pandas Dataframe
drought_df = pandas.DataFrame(drought_data)
# Convert the date column to a datatime object
drought_df["date"] = pandas.to_datetime(drought_df["date"])
# Sort the Dataframe by the date
drought_df = drought_df.sort_values("date")
drought_df

In [ ]:
# Save the pandas dataframes to the google drive
drought_df.to_excel(
    "/content/drive/MyDrive/drought_data_florida_mangroves/south_florida_mangroves_drought_data.xlsx", index=False
)
